<a href="https://colab.research.google.com/github/yunpiao/BPB-Worker-Panel/blob/main/SeedVR2_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler /content/ComfyUI/custom_nodes/ComfyUI-SeedVR2_VideoUpscaler

!pip install torchsde rotary-embedding-torch

!sed -i '/^[[:space:]]*from[[:space:]]\+server[[:space:]]\+import[[:space:]]\+PromptServer[[:space:]]*$/s/^/#/' /content/ComfyUI/custom_nodes/ComfyUI-SeedVR2_VideoUpscaler/src/interfaces/comfyui_node.py
!sed -i '/^[[:space:]]*PromptServer\.instance\.send_sync[[:space:]]*("progress"[[:space:]]*,[[:space:]]*progress_data[[:space:]]*,[[:space:]]*None)/s/^/#/' /content/ComfyUI/custom_nodes/ComfyUI-SeedVR2_VideoUpscaler/src/interfaces/comfyui_node.py

/content
Cloning into '/content/ComfyUI'...
remote: Enumerating objects: 27974, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 27974 (delta 47), reused 16 (delta 16), pack-reused 27901 (from 3)
Receiving objects: 100% (27974/27974), 68.68 MiB | 13.62 MiB/s, done.
Resolving deltas: 100% (18934/18934), done.
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-SeedVR2_VideoUpscaler'...
remote: Enumerating objects: 2421, done.
remote: Counting objects: 100% (612/612), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 2421 (delta 543), reused 473 (delta 473), pack-reused 1809 (from 3)
Receiving objects: 100% (2421/2421), 8.71 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (1578/1578), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.6 MB/s eta 0:00:00
sed: can't read /content/ComfyUI/custom_nodes/ComfyUI-SeedVR2_VideoUpscaler/src/interfaces/comfyui_node.py: No such file or directory
sed

In [ ]:
%cd /content/ComfyUI

import os, json, requests, random, time
from urllib.parse import urlsplit

import torch
from PIL import Image
import numpy as np

from nodes import NODE_CLASS_MAPPINGS, load_custom_node

load_custom_node("/content/ComfyUI/custom_nodes/ComfyUI-SeedVR2_VideoUpscaler")

SeedVR2 = NODE_CLASS_MAPPINGS["SeedVR2"]()
SeedVR2BlockSwap = NODE_CLASS_MAPPINGS["SeedVR2BlockSwap"]()
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()

def download_file(url, save_dir, file_name):
    os.makedirs(save_dir, exist_ok=True)
    file_suffix = os.path.splitext(urlsplit(url).path)[1]
    file_name_with_suffix = file_name + file_suffix
    file_path = os.path.join(save_dir, file_name_with_suffix)
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, 'wb') as file:
        file.write(response.content)
    return file_path

@torch.inference_mode()
def generate(input):
    try:
        values = input["input"]

        input_image = values['input_image']
        input_image = download_file(url=input_image, save_dir='/content/ComfyUI/input', file_name='input_image')
        seed = values['seed']
        new_resolution = values['new_resolution']
        batch_size = values['batch_size']
        preserve_vram = values['preserve_vram']

        if seed == 0:
                random.seed(int(time.time()))
                seed = random.randint(0, 2**32 - 1)

        input_image = LoadImage.load_image(input_image)[0]

        block_swap_config = SeedVR2BlockSwap.create_config(blocks_to_swap=32, use_non_blocking=True,
                            offload_io_components=False, cache_model=False, enable_debug=True)[0]

        sample = SeedVR2.execute(images=input_image, model="seedvr2_ema_3b_fp8_e4m3fn.safetensors", seed=seed, new_resolution=new_resolution,
                    batch_size=batch_size, preserve_vram=preserve_vram, block_swap_config=block_swap_config)[0]

        Image.fromarray(np.array(sample*255, dtype=np.uint8)[0]).save(f"output/flux_kontext_dev_output_image.png")

        result = Image.fromarray(np.array(sample*255, dtype=np.uint8)[0])

        return {"status": "DONE", "result": result}
    except Exception as e:
        return {"status": "ERROR", "result": str(e)}

In [ ]:
input = {
    "input": {
        "input_image": "https://d.uguu.se/PNZgwaHi.png",
        "seed": 0,
        "new_resolution": 1024,
        "batch_size": 1,
        "preserve_vram": False
    }
}

generate(input)["result"]